# Chatbot Sample

## Importing All Libraries that we are needed for the Chatbot

In [1]:
#importing nltk
import nltk 

#importing stemming library
from nltk.stem.lancaster import LancasterStemmer

#creating object for stemming
stemmer = LancasterStemmer()

#importing numpy library
import numpy

#importing keras library 
import keras

#importing tensorflow library 
import tensorflow

#importing random library
import random

#importing json library
import json

#importing regular expression array
import re

#stopwords helps us to remove the words like 'for', 'then', 'from', 'and' which are repeting again and again
#which does not put much value to identify sentance
from nltk.corpus import stopwords

#importing these keras libraries for further use
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from flask import Flask, jsonify, request
from tensorflow.keras.layers import LSTM

#importing pickle
import pickle

#Getting all the data from json file
with open("intents.json") as file:
    data = json.load(file)

#Printing the data for checking purposes
#print(data)
#TEST PASS


Using TensorFlow backend.


## Data Preprocessing

In [2]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, trainning, output = pickle.load(f)
        
except:

    #creating the list of all words in json file
    words = []

    #creating the list of all tags in json file
    labels = []

    #creating the list of all patterns in json file
    words_x = []

    #creating the list of all labels in json file
    labels_y = []

    #creating the list of tranning data
    trainning = []

    #creating the list of output format data
    output = []

    #STEMMING the process

    #loop through all the small small intents
    for intentPattern in data["intents"]:

        #loop through all the patterns in the small small intents
        for pattern in intentPattern["patterns"]:

            #replacing all other information like '. ,'  with space
            pattern = re.sub('[^a-zA-Z]',' ', pattern)

            #Getting all the words in the patterns
            wrds = nltk.word_tokenize(pattern)

            #List of the current pattern
            words_x.append(wrds)

            #appending the tags
            labels_y.append(intentPattern["tag"])

            #Stemming the words
            wrds = [stemmer.stem(word.lower()) for word in wrds]

            #storing the stemmerized words into list
            words.extend(wrds)

        #if label is not in intents we need to add that label
        if intentPattern["tag"] not in labels:
            labels.append(intentPattern["tag"])

    #Sorting the list and remove the uncommon atoms
    words = sorted(list(set(words)))

    #sorting the labels_y
    labels = sorted(labels)

    #Printing the words for checking purposes
    #print(words)
    #TEST PASS

    #Printing the labels for checking purposes
    #print(labels)
    #TEST PASS

    #creating list with all entry 0
    out_empty = [0 for _ in range(len(labels))]

    #Creating the Bag of Words model
    for x, doc in enumerate(words_x):

        # initialize our bag of words
        bag = []

        # stem each word - create base word, in attempt to represent related words
        wrds = [stemmer.stem(word.lower()) for word in doc]

        # create our bag of words array with 1, if word match found in current pattern
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        trainning.append(bag)

        # output is a '0' for each tag and '1' for current tag (for each pattern)
        output_row = out_empty[:]
        output_row[labels.index(labels_y[x])] = 1
        output.append(output_row)

    #Printing the words for checking purposes
    #print(trainning)
    #TEST PASS

    #Printing the labels for checking purposes
    #print(output)
    #TEST PASS

    #converting the trainning list to numpy array
    trainning = numpy.array(trainning)

    #converting the output list to numpy array
    output = numpy.array(output)
    
    #used pickle to load previous output
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, trainning, output), f)

## Finishing Touch to Data

In [3]:
## Storing trainning values to train_x
train_x = trainning

#Chceking the shap of train_X
print(train_x.shape[0])

#Storing trainning values to train_y
train_y = output

#Chceking the shap of train_y
print(train_y.shape[0])

86
86


## Model Building

In [4]:
#create model
model = Sequential()

#get number of columns in training data
n_cols = len(train_x[0])

# Create model - 3 layers. First layer 128 neurons, 
# Second layer 64 neurons and 
# Third output layer contains number of neurons
# equal to number of intents to predict output intent with softmax

#add model layers
model.add(Dense(128, activation='relu', input_shape=(n_cols,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

#compile model using mse as a measure of model performance
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

## Load The Previously Load Model

In [5]:
#try and load previously trained model
try:
    print("Model Load Successfully")
    model.load("model.keras")
except:
    # Fit the model
    model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
    model.save("model.keras")
    print("Model Saved Successfully")

Model Load Successfully
Epoch 1/200
86/86 [==============================] - 3s 35ms/step - loss: 2.7977 - accuracy: 0.0814
Epoch 2/200
86/86 [==============================] - 0s 336us/step - loss: 2.4363 - accuracy: 0.2209
Epoch 3/200
86/86 [==============================] - 0s 695us/step - loss: 2.1920 - accuracy: 0.3140
Epoch 4/200
86/86 [==============================] - 0s 510us/step - loss: 1.9221 - accuracy: 0.3721
Epoch 5/200
86/86 [==============================] - 0s 464us/step - loss: 1.6218 - accuracy: 0.5581
Epoch 6/200
86/86 [==============================] - 0s 406us/step - loss: 1.3046 - accuracy: 0.6860
Epoch 7/200
86/86 [==============================] - 0s 441us/step - loss: 1.2859 - accuracy: 0.5581
Epoch 8/200
86/86 [==============================] - 0s 348us/step - loss: 1.1215 - accuracy: 0.6047
Epoch 9/200
86/86 [==============================] - 0s 475us/step - loss: 0.9655 - accuracy: 0.7326
Epoch 10/200
86/86 [==============================] - 0s 568us/step 

86/86 [==============================] - 0s 475us/step - loss: 0.1133 - accuracy: 0.9767
Epoch 82/200
86/86 [==============================] - 0s 510us/step - loss: 0.0337 - accuracy: 0.9767
Epoch 83/200
86/86 [==============================] - 0s 591us/step - loss: 0.0495 - accuracy: 1.0000
Epoch 84/200
86/86 [==============================] - 0s 510us/step - loss: 0.0536 - accuracy: 0.9767
Epoch 85/200
86/86 [==============================] - 0s 580us/step - loss: 0.0266 - accuracy: 1.0000
Epoch 86/200
86/86 [==============================] - 0s 464us/step - loss: 0.0629 - accuracy: 0.9884
Epoch 87/200
86/86 [==============================] - 0s 731us/step - loss: 0.0245 - accuracy: 1.0000
Epoch 88/200
86/86 [==============================] - 0s 487us/step - loss: 0.0608 - accuracy: 0.9884
Epoch 89/200
86/86 [==============================] - 0s 615us/step - loss: 0.0639 - accuracy: 0.9884
Epoch 90/200
86/86 [==============================] - 0s 928us/step - loss: 0.0700 - accuracy: 

86/86 [==============================] - 0s 626us/step - loss: 0.0468 - accuracy: 0.9767
Epoch 161/200
86/86 [==============================] - 0s 754us/step - loss: 0.0800 - accuracy: 0.9651
Epoch 162/200
86/86 [==============================] - 0s 603us/step - loss: 0.0235 - accuracy: 0.9884
Epoch 163/200
86/86 [==============================] - 0s 464us/step - loss: 0.0905 - accuracy: 0.9767
Epoch 164/200
86/86 [==============================] - 0s 522us/step - loss: 0.0174 - accuracy: 1.0000
Epoch 165/200
86/86 [==============================] - 0s 823us/step - loss: 0.0459 - accuracy: 0.9651
Epoch 166/200
86/86 [==============================] - 0s 835us/step - loss: 0.0274 - accuracy: 0.9884
Epoch 167/200
86/86 [==============================] - 0s 823us/step - loss: 0.0238 - accuracy: 0.9884
Epoch 168/200
86/86 [==============================] - 0s 626us/step - loss: 0.0083 - accuracy: 1.0000
Epoch 169/200
86/86 [==============================] - 0s 487us/step - loss: 0.0120 - a

In [6]:
#Print the accuracy of model
scores = model.evaluate(train_x, train_y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 100.00%


## Predicting the data. 

## Step 1:- Creating Bag of Words of Input Received

In [7]:
#First Create a bag of Words of Input string
def bag_of_words(sentence, words):
    
    # initialize our bag of words
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    
    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
    
    return numpy.array(bag)

## Step 2:- Taking Input From User

In [8]:
#Creating function for chatting
def chat():
    print("Start talking with Bot now (type 'quit' to stop)!!!")
    
    while True:
        input_user = input("You: ")
        
        if input_user.lower() == "quit":
            break
        
        input_data = [bag_of_words(input_user, words)]
        
        #Predict the model
        results = model.predict([input_data])[0]
        
        result_index = numpy.argmax(results)
        
        tag = labels[result_index]
        
        if results[result_index]>0.7:
            for tg in data["intents"]:
                if tg["tag"] == tag:
                    response = tg['responses']            
                    print(random.choice(response))
        else:            
            print("Sorry, can't understand you, Please give me more info")        
        
#         ERROR_THRESHOLD = 0.25
#         results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
        
#         # sort by strength of probability
#         results.sort(key=lambda x: x[1], reverse=True)

#         tag = labels[results[0][0]]
        
#             for tg in data["intents"]:
#                 if tg['tag'] == tag
#                     response = tg['responses']            
#                     print(random.choice(response))
        
chat()

Start talking with Bot now (type 'quit' to stop)!!!
You: hi
Hey, How can I help you?
You: quit
